In [20]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=[ "ner","tagger",'textcat'])
from spellchecker import SpellChecker
import nltk
from nltk.tokenize import word_tokenize
import re
import string
import unicodedata
import emoji as emoji
import regex
import spacy
from spacy.tokens import Token
from bs4 import BeautifulSoup
import pandas as pd
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pandas import Panel
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold
import time
import numpy as np
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
import warnings
warnings.filterwarnings("ignore")

In [21]:

def SVC_grid_search(hyper_comb,X_train,y_train,file_name):
    start=time.time()
    all_res_cv = []
    cv = KFold(n_splits=5, shuffle=True, random_state=1848773) # 5 fold
    grid = ParameterGrid(hyper_comb) #all possible combination of parameters chosen
    for hypers in tqdm(grid):
        val_f1_list = []
        train_f1_list = []
        val_recall_list=[]
        train_recall_list=[]
        val_accuracy_list=[]
        train_accuracy_list=[]
        val_pre_list=[]
        train_pre_list=[]
        C = hypers['C']
        coef0 = hypers['coef0']
        kernel=hypers['kernel']
        degree=hypers['degree']
        gamma=hypers['gamma']
        shrinking=hypers['shrinking']
        decision_function_shape=hypers['decision_function_shape']

        for train_index, test_index in cv.split(X_train): # cross validation
            #take 80% and 20%
            X_train_cv, X_val, y_train_cv, y_val = X_train[train_index], X_train[test_index], y_train[train_index], y_train[test_index]
            #fit the model with the chosen parameters
            model=SVC(C=C,coef0=coef0,kernel=kernel,degree=degree,gamma=gamma,shrinking=shrinking,decision_function_shape=decision_function_shape,random_state=1848773).fit(X_train_cv,y_train_cv)
            # make prediction and compute F1
            pred_train = model.predict(X_train_cv)
            pred_val = model.predict(X_val)
            #F1 SCORE
            f1_train = f1_score(y_train_cv, pred_train, average='micro')
            train_f1_list.append(f1_train)
            f1_val = f1_score(y_val, pred_val, average='micro')
            val_f1_list.append(f1_val)
            #RECALL VALUES
            recall_train=recall_score(y_train_cv,pred_train, average='micro')
            train_recall_list.append(recall_train)
            recall_val=recall_score(y_val,pred_val, average='micro')
            val_recall_list.append(recall_val)
            #ACCURACY
            accuracy_train=accuracy_score(y_train_cv,pred_train)
            train_accuracy_list.append(accuracy_train)
            accuracy_val=accuracy_score(y_val,pred_val)
            val_accuracy_list.append(accuracy_val)
            #PRECISION
            pre_train=precision_score(y_train_cv,pred_train, average='micro')
            train_pre_list.append(pre_train)
            pre_val=precision_score(y_val,pred_val, average='micro')
            val_pre_list.append(pre_val)
            
        all_res_cv.append({'C':C,
                           'coef0' :coef0,
                            'kernel':kernel,
                            'degree':degree,
                            'gamma':gamma,
                        'shrinking':shrinking,
                            'decision_function_shape':decision_function_shape,

                           'val_f1':np.mean(val_f1_list), 
                           'train_f1':np.mean(train_f1_list),
                           'val_recall':np.mean(val_recall_list), 
                           'train_recall':np.mean(train_recall_list),
                           'val_acc':np.mean(val_accuracy_list), 
                           'train_acc':np.mean(train_accuracy_list),
                           'val_pre':np.mean(val_pre_list),
                          'train_pre':np.mean(train_pre_list)})
        
        '''
        print(({'C':C,
                           'coef0' :coef0,
                            'kernel':kernel,
                            'degree':degree,
                            'gamma':gamma,
                        'shrinking':shrinking,
                            'decision_function_shape':decision_function_shape,'val_f1':np.mean(val_f1_list), 'train_f1':np.mean(train_f1_list)}))
        '''

        # after the 5th iteration we make the mean of the collected errors to find a more reliable value for the error
    end=time.time()-start
    df = pd.DataFrame(all_res_cv) # collect all results in a dataframe
    df.to_csv(file_name, index=False) # save dataframe in csv with results

    return print('File created grid_search_cv')


In [22]:
train=pd.read_csv('pre_train.csv')

In [23]:
sub=list(train['prep_text'])
sub[5115]='Err:509'
train['prep_text']=sub

## bow_models

In [24]:
X = train['prep_text']
y = train['target']

In [25]:
count_vectorizer = feature_extraction.text.CountVectorizer()
X_train_=count_vectorizer.fit_transform(X)

In [33]:
hyper_comb={'C':list(np.arange(0.5,3,0.25)),
            'coef0':[1],
              'kernel':['linear','poly','rbf','sigmoid'],
           'degree':[3],
           'gamma':['scale','auto'],
           'shrinking':[True,False],
           'decision_function_shape':['ovo','ovr']} ##combination of hyperparameters

SVC_grid_search(hyper_comb,X_train_,y,'SVC_grid_search_cv_bow.csv')

In [35]:
max(list(pd.read_csv('svc_grid_search_cv_bow.csv')['val_f1']))

0.8048080979945695

### tf_idf

In [38]:
tfidf_transformer = TfidfTransformer()
train_tfidf_ = tfidf_transformer.fit_transform(X_train_)

SVC_grid_search(hyper_comb,train_tfidf_,y,'SVC_grid_search_cv_tfidf.csv')

In [40]:
max(list(pd.read_csv('svc_grid_search_cv_tfidf.csv')['val_f1']))

0.8024433068766863